In [1]:
## in this notebook we are taking the prepared data from previous version and for model building we willstack different models.

In [108]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score , f1_score , precision_score , recall_score  

In [7]:
train = pd.read_csv(r"../Data/PreparedData/train.csv")
test = pd.read_csv(r"../Data/PreparedData/test.csv")

In [20]:
X = train.drop("Survived" , axis = 1).values
y = train["Survived"].values

In [41]:
x_train_org , x_test_org, y_train_org  , y_test_org = train_test_split(X , y , test_size = 0.3 , random_state = 42)

In [42]:
n_train_org = x_train_org.shape[0]
n_test_org = x_test_org.shape[0]
NFOLDS = 4

In [43]:
kf = KFold(n_splits=NFOLDS , shuffle = False , random_state = 42)

C:\Users\nihal\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [87]:
class SklearnHelper(object):
    def __init__(self , clf , seed =0, params = {}):
        params["random_state"] = seed
        self.clf = clf(**params)
        
        
    def fit(self , x_train ,y_train):
        return self.clf.fit(x_train , y_train)
    
    def Predict(self, x_test , y_test):
        return self.clf.predict(x_test )
    
    def FeatureImportance(self ,x_train , y_train):
        return self.clf.fit(x_train , y_train).feature_importances
    
        

In [89]:
SEED = 42
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED)
svc = SklearnHelper(clf=SVC, seed=SEED)

In [72]:
def oob_generator(clf, x_train_org , y_train_org , x_test_org):

    oob_train = np.zeros((n_train_org, ))
    oob_test_fld = np.zeros((n_test_org , NFOLDS))
    oob_test = np.zeros((n_test_org , ))

    for i ,(train_i ,test_i) in enumerate(kf.split(x_train_org)):
        x_train, y_train = x_train_org[train_i] , y_train_org[train_i]
        x_test , y_test = x_train_org[test_i] , y_train_org[test_i]

        clf.fit(x_train , y_train)

        oob_train[test_i] = Dtree.predict(x_test)
        oob_test_fld[: , i] = Dtree.predict(x_test_org)

    oob_test = oob_test_fld.mean(axis = 1)

    return oob_train , oob_test

In [114]:
# oob_test

In [90]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = oob_generator(et, x_train_org, y_train_org, x_test_org) # Extra Trees
rf_oof_train, rf_oof_test = oob_generator(rf,x_train_org, y_train_org, x_test_org) # Random Forest
ada_oof_train, ada_oof_test = oob_generator(ada, x_train_org, y_train_org, x_test_org) # AdaBoost 
gb_oof_train, gb_oof_test = oob_generator(gb,x_train_org, y_train_org, x_test_org) # Gradient Boost
svc_oof_train, svc_oof_test = oob_generator(svc,x_train_org, y_train_org, x_test_org) # Support Vector Classifier

In [98]:
IInd_train = pd.DataFrame({"RF":rf_oof_train , "et" : et_oof_train , "ada" : ada_oof_train , "gb" : gb_oof_train , "svc" : svc_oof_train})

In [107]:
IInd_test = pd.DataFrame({"RF":rf_oof_test , "et" : et_oof_test , "ada" : ada_oof_test , "gb" : gb_oof_test , "svc" : svc_oof_test})

In [99]:
IInd_train

,RF,et,ada,gb,svc
0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
618,0.0,0.0,0.0,0.0,0.0
619,0.0,0.0,0.0,0.0,0.0
620,0.0,0.0,0.0,0.0,0.0
621,0.0,0.0,0.0,0.0,0.0


In [103]:
import xgboost as xgb
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)

In [106]:
gbm.fit(IInd_train , y_train_org)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.9, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [110]:
accuracy_score(y_test_org , gbm.predict(IInd_test))

0.7723880597014925

In [111]:
f1_score(y_test_org , gbm.predict(IInd_test))

0.6995073891625616

In [112]:
precision_score(y_test_org , gbm.predict(IInd_test))

0.7717391304347826

In [113]:
recall_score(y_test_org , gbm.predict(IInd_test))

0.6396396396396397

In [177]:
# x_train_org , x_test_org, y_train_org  , y_test_org = train_test_split(X , y , test_size = 0.3 , random_state = 42)

train = pd.read_csv(r"../Data/PreparedData/train.csv")
test = pd.read_csv(r"../Data/PreparedData/test.csv")

In [178]:
x_train_org = train.drop("Survived" , axis = 1).values
y_train_org = train["Survived"].values
x_test_org = test.values

In [179]:
n_train_org = x_train_org.shape[0]
n_test_org = x_test_org.shape[0]
NFOLDS = 4

In [180]:
kf = KFold(n_splits=NFOLDS , shuffle = False , random_state = 42)

C:\Users\nihal\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [181]:
class SklearnHelper(object):
    def __init__(self , clf , seed =0, params = {}):
        params["random_state"] = seed
        self.clf = clf(**params)
        
        
    def fit(self , x_train ,y_train):
        return self.clf.fit(x_train , y_train)
    
    def Predict(self, x_test , y_test):
        return self.clf.predict(x_test )
    
    def FeatureImportance(self ,x_train , y_train):
        return self.clf.fit(x_train , y_train).feature_importances
    
        

In [182]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [163]:
SEED = 42
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [164]:
n_train_org = x_train_org.shape[0]
n_test_org = x_test_org.shape[0]
NFOLDS = 4

def oob_generator(clf, x_train_org , y_train_org , x_test_org):

    oob_train = np.zeros((n_train_org, ))
    oob_test_fld = np.zeros((n_test_org , NFOLDS))
    oob_test = np.zeros((n_test_org , ))

    for i ,(train_i ,test_i) in enumerate(kf.split(x_train_org)):
#         print("train_i->" , train_i)
#         print("test_i->" , test_i)
        
        x_train, y_train = x_train_org[train_i] , y_train_org[train_i]
        x_test , y_test = x_train_org[test_i] , y_train_org[test_i]

        clf.fit(x_train , y_train)

        oob_train[test_i] = Dtree.predict(x_test)
        oob_test_fld[: , i] = Dtree.predict(x_test_org)

    oob_test = oob_test_fld.mean(axis = 1)

    return oob_train , oob_test

In [165]:
# oob_test

In [166]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = oob_generator(et, x_train_org, y_train_org, x_test_org) # Extra Trees
rf_oof_train, rf_oof_test = oob_generator(rf,x_train_org, y_train_org, x_test_org) # Random Forest
ada_oof_train, ada_oof_test = oob_generator(ada, x_train_org, y_train_org, x_test_org) # AdaBoost 
gb_oof_train, gb_oof_test = oob_generator(gb,x_train_org, y_train_org, x_test_org) # Gradient Boost
svc_oof_train, svc_oof_test = oob_generator(svc,x_train_org, y_train_org, x_test_org) # Support Vector Classifier

C:\Users\nihal\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:368: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\Users\nihal\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:368: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
C:\Users\nihal\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:368: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [167]:
IInd_train = pd.DataFrame({"RF":rf_oof_train , "et" : et_oof_train , "ada" : ada_oof_train , "gb" : gb_oof_train , "svc" : svc_oof_train})

In [168]:
IInd_test = pd.DataFrame({"RF":rf_oof_test , "et" : et_oof_test , "ada" : ada_oof_test , "gb" : gb_oof_test , "svc" : svc_oof_test})

In [169]:
IInd_train

,RF,et,ada,gb,svc
0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
886,0.0,0.0,0.0,0.0,0.0
887,1.0,1.0,1.0,1.0,1.0
888,0.0,0.0,0.0,0.0,0.0
889,0.0,0.0,0.0,0.0,0.0


In [170]:
import xgboost as xgb
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)

In [171]:
gbm.fit(IInd_train , y_train_org)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.9, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [172]:
pred =  gbm.predict(IInd_test)

In [173]:
subm = pd.read_csv(r"../Data/gender_submission.csv")

In [174]:
subm.columns

Index(['PassengerId', 'Survived'], dtype='object')

In [175]:
submitDF = pd.DataFrame({"PassengerId" : subm["PassengerId"] , "Survived" : pred})

In [176]:
submitDF.to_csv(r"../Data/Predictions/TunedStackXG.csv" , index = False)